# Getting deeper with Keras
* Tensorflow is a powerful and flexible tool, but coding large neural architectures with it is tedious.
* There are plenty of deep learning toolkits that work on top of it like Slim, TFLearn, Sonnet, Keras.
* Choice is matter of taste and particular task
* We'll be using Keras

In [ ]:
import sys
sys.path.append("..")
import grading

In [ ]:
email = ''
token = ''

In [ ]:
# use preloaded keras datasets and models
from pathlib import Path
cur_dir = Path('.').absolute()

src_keras = cur_dir.parent.joinpath('readonly','keras')

if not src_keras.is_dir():
    raise RuntimeError('readonly directory not found\n'
                       'Download it by running download_resources.ipynb in the root dir of the repository')

src_datasets = src_keras.joinpath('datasets')
src_models = src_keras.joinpath('models')

home = cur_dir.home()
dst_datasets = home.joinpath('.keras', 'datasets')
dst_models = home.joinpath('.keras', 'models')
dst_datasets.mkdir(exist_ok=True, parents=True)
dst_models.mkdir(exist_ok=True, parents=True)

In [ ]:
src_dataset_files = src_datasets.glob('*')
for f in src_dataset_files:
    if not f.is_file():
        name = f.name
        dst = dst_datasets.joinpath(name)
        dst.symlink_to(f)
        dst.write_bytes(dst.read_bytes())

src_models_files = src_models.glob('*')
for f in src_models_files:
    if not f.is_file():
        name = f.name
        dst = dst_models.joinpath(name)
        dst.symlink_to(f)
        dst.write_bytes(dst.read_bytes())

In [ ]:
import numpy as np
from preprocessed_mnist import load_dataset
import keras
X_train, y_train, X_val, y_val, X_test, y_test = load_dataset()
y_train,y_val,y_test = map(keras.utils.np_utils.to_categorical,[y_train,y_val,y_test])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(X_train[0]);

## The pretty keras

In [ ]:
import tensorflow as tf
s = tf.InteractiveSession()

In [ ]:
import keras
from keras.models import Sequential
import keras.layers as ll

model = Sequential(name="mlp")

model.add(ll.InputLayer([28, 28]))

model.add(ll.Flatten())

# network body
model.add(ll.Dense(500, activation='relu'))

# output layer: 10 neurons for each class with softmax
model.add(ll.Dense(10, activation='softmax'))

# categorical_crossentropy is your good old crossentropy
# but applied for one-hot-encoded vectors
model.compile("adam", "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
model.summary()

### Model interface

Keras models follow __Scikit-learn__'s interface of fit/predict with some notable extensions. Let's take a tour.

In [ ]:
# fit(X,y) ships with a neat automatic logging.
#          Highly customizable under the hood.
model.fit(X_train, y_train,
          validation_data=(X_val, y_val), epochs=30);

In [ ]:
# estimate probabilities P(y|x)
model.predict_proba(X_val[:2])

In [ ]:
# Save trained weights
model.save("weights.h5")

In [ ]:
print("\nLoss, Accuracy = ", model.evaluate(X_test, y_test))

### Whoops!
So far, our model is staggeringly inefficient. There is something wrong with it. Guess, what?

In [ ]:
# Test score...
test_predictions = model.predict_proba(X_test).argmax(axis=-1)
test_answers = y_test.argmax(axis=-1)

test_accuracy = np.mean(test_predictions==test_answers)

print("\nTest accuracy: {} %".format(test_accuracy*100))

assert test_accuracy>=0.92,"Logistic regression can do better!"
assert test_accuracy>=0.975,"Your network can do better!"
print("Great job!")

In [ ]:
answer_submitter = grading.Grader("0ybD9ZxxEeea8A6GzH-6CA")
answer_submitter.set_answer("N56DR", test_accuracy)

In [ ]:
answer_submitter.submit(email, token)

## Keras + tensorboard

Remember the interactive graphs from Tensorboard one notebook ago? 

Thing is, Keras can use tensorboard to show you a lot of useful information about the learning progress. Just take a look!

In [ ]:
import shutil
tboard_dir = Path(Path('.').absolute().root).joinpath('tmp', 'tboard')
shutil.rmtree(tboard_dir, ignore_errors=True)

In [ ]:
from keras.callbacks import TensorBoard
model.fit(X_train, y_train, validation_data=(X_val, y_val), 
          epochs=10,
          callbacks=[TensorBoard("/tmp/tboard")])

# Tips & tricks

Here are some tips on what you could do. Don't worry, to reach the passing threshold you don't need to try all the ideas listed here, feel free to stop once you reach the 0.975 accuracy mark.

 * __Network size__
   * More neurons, 
   * More layers, ([docs](https://keras.io/))

   * Nonlinearities in the hidden layers
     * tanh, relu, leaky relu, etc
   * Larger networks may take more epochs to train, so don't discard your net just because it could didn't beat the baseline in 5 epochs.


 * __Early Stopping__
   * Training for 100 epochs regardless of anything is probably a bad idea.
   * Some networks converge over 5 epochs, others - over 500.
   * Way to go: stop when validation score is 10 iterations past maximum
     

 * __Faster optimization__
   * rmsprop, nesterov_momentum, adam, adagrad and so on.
     * Converge faster and sometimes reach better optima
     * It might make sense to tweak learning rate/momentum, other learning parameters, batch size and number of epochs


 * __Regularize__ to prevent overfitting
   * Add some L2 weight norm to the loss function, theano will do the rest
     * Can be done manually or via - https://keras.io/regularizers/
   
   
 * __Data augmemntation__ - getting 5x as large dataset for free is a great deal
   * https://keras.io/preprocessing/image/
   * Zoom-in+slice = move
   * Rotate+zoom(to remove black stripes)
   * any other perturbations
   * Simple way to do that (if you have PIL/Image): 
     * ```from scipy.misc import imrotate,imresize```
     * and a few slicing
   * Stay realistic. There's usually no point in flipping dogs upside down as that is not the way you usually see them.